# KMeans to test multiple dimensions at a time (gender)

## 0. Data Loading : dataset of NOUN + ADJ

In [1]:
import pandas as pd

# WE loading
all_n_we = pd.read_csv('../Data/FlauBERT_WE/all_nouns_we.csv', index_col=0).drop(columns=["number"])
all_a_we = pd.read_csv('../Data/FlauBERT_WE/all_adjectives_we.csv', index_col=0).drop(columns = ["number"])
all_na_we = pd.concat([all_n_we, all_a_we])

# Normalization: values fall within -1 and 1
for i in range(512):

    values, copy = list(all_na_we[str(i)]), list(all_na_we[str(i)])
    copy.sort()
    min_, max_ = copy[0], copy[len(copy)-1]
    all_na_we[str(i)] = [val / (max_ - min_) for val in values]

all_na_we["gender"] = all_na_we["gender"].apply(lambda x: 1 if x == "masculine" else 0)

In [2]:
import sys
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

## Testing 1 Dimension at a time  
execution time ~21sec.

In [3]:
dim_list_1= [str(i) for i in range(512)] # all dimensions
df_clustering_1 = pd.DataFrame(columns = ["ARI"])
ari = []

for dim in dim_list_1:

    # Clustering using Kmeans
    km = KMeans(n_clusters=2, random_state=42, n_init=10)
    km.fit(all_na_we[dim].values.reshape(-1, 1))

    # Compute the Adjusted Rand Index: the closer to 1, the better
    ari.append(adjusted_rand_score(all_na_we["gender"], km.labels_))

df_clustering_1["ARI"] = ari
df_clustering_1.index = dim_list_1

In [4]:
df_clustering_1.sort_values(by = "ARI", ascending = False).head(10)

,ARI
245,0.027971
192,0.025293
121,0.023947
507,0.023088
5,0.020263
250,0.019493
470,0.019037
439,0.018981
276,0.018728
133,0.018617


## Testing 2 dimensions at a time

In [8]:
dim_list_2 = [[str(i), str(j)] for i in range(512) for j in range(512) if i < j] # all pairs of dimensions
print("Number of combinations tested: ", len(dim_list_2))

df_clustering_2 = pd.DataFrame(columns = ["ARI"])
ari = []


for i, dim in enumerate(dim_list_2):

    # Clustering using Kmeans
    km = KMeans(n_clusters=2, random_state=42, n_init=10)
    km.fit(all_na_we[dim])

    # Compute the Adjusted Rand Index: the closer to 1, the better
    ari.append(adjusted_rand_score(all_na_we["gender"], km.labels_))

df_clustering_2["ARI"] = ari
df_clustering_2.index = dim_list_2

130816
.
.
.


KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 177, in where
KeyboardInterrupt: 
